In [4]:
import pandas as pd
from geopy.geocoders import Nominatim
from datetime import date

In [5]:
ds = 'CURBToolEUIDataset.xlsx'
multipliers = 'emissionsmultipliers_sample.csv'
source = 'CURB Tool'

country_of_interest = 'Mexico'
country_specific_eui = 'MX_EUI_CURB_estimates.xlsx'

In [12]:
def get_geocode(city):
    # initialize Nominatim API
    geolocator = Nominatim(user_agent="MyApp")
    gc = geolocator.geocode(city)
    return gc


def city_to_country(city): 
    gc = get_geocode(city)
    gc_arr = str(gc).split(", ")
    return gc_arr[-1]


def city_to_region(city): 
    gc = get_geocode(city)
    gc_arr = str(gc).split(", ")
    if len(gc_arr) == 1: return gc_arr[-1]
    elif gc_arr[-2].isdigit(): return gc_arr[-3]
    return gc_arr[-2]


def city_to_coords(city): 
    gc = get_geocode(city)
    if gc == None: return [None, None]
    return [gc.latitude, gc.longitude]


def city_to_eui(city, df):
    city_df = df.loc[df['City'] == city]
    print(city_df)
    return 

def find_multipliers(city, df, euis, countrydf, residential=False):
    city_multipliers = cseui.loc[cseui['City'] == city]
    #city_df = df.loc[df['City'] == city]
    if residential == True:
        print(city_multipliers['Energy Use Intensity by End Use (kwh/m2/year)'])
    else:
        print(city_multipliers['Energy Use Intensity by End Use (kwh/m2/year)2'])
    print(city_multipliers)
    return 

def city_to_res_mult(city, resmults):
    return resmults[city]

def city_to_comm_mult(city, commmults):
    return commmults[city]

def get_citylist(df, source, resmults, commmults):
    cities = list(set(df['City'].tolist()))
    curr_date = str(date.today())
    #for city in cities:
    #    print(float(city_to_res_mult(city, euis)))
    data = {
        'City': cities,
        'Region': list(map(city_to_region, cities)),
        'Country': list(map(city_to_country, cities)),
        'Latitude': list(map(lambda city: city_to_coords(city)[0], cities)),
        'Longitude': list(map(lambda city: city_to_coords(city)[1], cities)),
        'Residential_Multiplier': list(map(lambda city: resmults[city], cities)),
        'Commercial_Multiplier': list(map(lambda city: commmults[city], cities)),
        'Source': [source] * len(cities),
        'Acquisition_Date': [curr_date] * len(cities),
        'Notes': [''] * len(cities)
    }
    cities_df = pd.DataFrame(data)
    return cities_df.dropna()

In [13]:
df = pd.read_excel(ds)
multdf = pd.read_csv(multipliers)

df_top = df.head() 
df_top

,City,Country,Primary,Secondary,Tertiary,Building Type,Heating Energy,Cooling Energy,Fan Energy,Pump Energy,...,Catering,"Equipment, Lift….",Computers,Refrigeration,Home Appliances,Common Amenities,Total,Floor Area,Source,Energy Use Intensity by End Use by City (kwh/m2/year)
0,Abu Dhabi,United Arab Emirates,Homes,Low,Flats/Apartments,Res Low Flat,1.584393,42.880124,10.477154,0.0,...,0.0,0.0,0.0,0.0,34.179712,6.033327,135.392032,366.021341,DOE Model,Scale: City
1,Abu Dhabi,United Arab Emirates,Homes,Low,House,Res Low House,4.632241,46.318026,11.033556,0.0,...,0.0,0.0,0.0,0.0,21.920461,5.431969,121.503766,732.045732,DOE Model,NaN
2,Abu Dhabi,United Arab Emirates,Homes,Lower Middle,Flats/Apartments,Res Low Mid Flat,1.584393,42.880124,10.477154,0.0,...,0.0,0.0,0.0,0.0,34.179712,6.033327,135.392032,366.021341,DOE Model,NaN
3,Abu Dhabi,United Arab Emirates,Homes,Lower Middle,House,Res Low Mid House,4.632241,46.318026,11.033556,0.0,...,0.0,0.0,0.0,0.0,21.920461,5.431969,121.503766,732.045732,DOE Model,NaN
4,Abu Dhabi,United Arab Emirates,Homes,Upper Middle,Flats/Apartments,Res High Mid Flat,1.584393,42.880124,10.477154,0.0,...,0.0,0.0,0.0,0.0,34.179712,6.033327,135.392032,366.021341,DOE Model,NaN


In [14]:
multdf

,nation,state,city,residential_multiplier,commercial_multiplier
0,usa,massachusetts,boston,193.84,190.07
1,usa,michigan,chicago,194.35,223.76
2,usa,mexico,mexico city,106.03,94.93
3,usa,indonesia,bali,69.06,69.06


In [15]:
cseui = pd.read_excel(country_specific_eui).dropna()
cseui_top = cseui.head() 
cseui_top
cs_res_multipliers = dict(zip(cseui['City'], cseui['Energy Use Intensity by End Use (kwh/m2/year)']))
cs_comm_multipliers = dict(zip(cseui['City'], cseui['Energy Use Intensity by End Use (kwh/m2/year)2']))

In [16]:
multdf
countrydf = df[df['Country'] == country_of_interest]
countrydf

,City,Country,Primary,Secondary,Tertiary,Building Type,Heating Energy,Cooling Energy,Fan Energy,Pump Energy,...,Catering,"Equipment, Lift….",Computers,Refrigeration,Home Appliances,Common Amenities,Total,Floor Area,Source,Energy Use Intensity by End Use by City (kwh/m2/year)
3366,Aerop Internacional,Mexico,Homes,Low,Flats/Apartments,Res Low Flat,0.00,20.689655,2.0,0.0,...,0.0,0.0,0.0,0.0,12.0,4.0,74.689655,50.0,EDGE,NaN
3367,Aerop Internacional,Mexico,Homes,Low,House,Res Low House,1.25,61.379310,7.0,0.0,...,0.0,0.0,0.0,0.0,8.0,3.0,108.629310,75.0,EDGE,NaN
3368,Aerop Internacional,Mexico,Homes,Lower Middle,Flats/Apartments,Res Low Mid Flat,0.00,16.551724,2.0,0.0,...,0.0,0.0,0.0,0.0,14.0,3.0,66.551724,50.0,EDGE,NaN
3369,Aerop Internacional,Mexico,Homes,Lower Middle,House,Res Low Mid House,1.25,50.689655,5.0,0.0,...,0.0,0.0,0.0,0.0,10.0,2.0,93.439655,100.0,EDGE,NaN
3370,Aerop Internacional,Mexico,Homes,Upper Middle,Flats/Apartments,Res High Mid Flat,0.00,15.172414,2.0,0.0,...,0.0,0.0,0.0,0.0,25.0,3.0,88.172414,50.0,EDGE,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5893,Mexico City,Mexico,Hospital,Public Hospital,0,Hospital,25.00,1.333333,8.0,1.0,...,29.0,63.0,0.0,0.0,0.0,0.0,203.333333,10000.0,EDGE,NaN
5894,Monterrey,Mexico,Hospital,Public Hospital,0,Hospital,9.00,3.877551,13.0,4.0,...,29.0,63.0,0.0,0.0,0.0,0.0,189.877551,10000.0,EDGE,NaN
5895,San Luis Potosí,Mexico,Hospital,Public Hospital,0,Hospital,26.00,1.777778,8.0,1.0,...,29.0,63.0,0.0,0.0,0.0,0.0,203.777778,10000.0,EDGE,NaN
5896,Tampico,Mexico,Hospital,Public Hospital,0,Hospital,2.00,4.285714,13.0,4.0,...,29.0,63.0,0.0,0.0,0.0,0.0,181.285714,10000.0,EDGE,NaN


In [17]:
citylist = get_citylist(countrydf, source, cs_res_multipliers, cs_comm_multipliers)

In [18]:
citylist

,City,Region,Country,Latitude,Longitude,Residential_Multiplier,Commercial_Multiplier,Source,Acquisition_Date,Notes
0,Guanajuato,Guanajuato,México,20.987700,-101.000000,27.50000,16.165414,CURB Tool,2023-09-19,
1,Tampico,Tamaulipas,México,22.215762,-97.857771,15.15625,8.819549,CURB Tool,2023-09-19,
3,Hermosillo,Sonora,México,29.094821,-110.969220,37.34375,18.071429,CURB Tool,2023-09-19,
4,Mexicali,Baja California,México,32.640525,-115.474899,32.81250,15.851504,CURB Tool,2023-09-19,
5,Mexico City,Ciudad de México,México,19.432630,-99.133178,41.09375,22.979323,CURB Tool,2023-09-19,
6,Mérida,Extremadura,España,38.917467,-6.344398,13.28125,6.693609,CURB Tool,2023-09-19,
7,Veracruz,Veracruz,México,19.333333,-96.666667,14.37500,7.475564,CURB Tool,2023-09-19,
8,San Luis Potosí,San Luis Potosí,México,22.500000,-100.494914,47.65625,23.479323,CURB Tool,2023-09-19,
9,Cancun,Quintana Roo,México,21.161785,-86.851047,12.03125,6.943609,CURB Tool,2023-09-19,
10,Mazatlán,Sinaloa,México,23.231305,-106.415314,15.15625,8.913534,CURB Tool,2023-09-19,


In [21]:
import json

geojson = {"type": "FeatureCollection", "features": []}

for _, row in citylist.iterrows():
    feature = {
        "type": "Feature", 
        "geometry": {
            "type": "Point", 
            "coordinates": [row['Longitude'], row['Latitude']]}, 
        "properties": {
            "city": row['City'], 
            "region": row['Region'],
            "country": row['Country'],
            "residential_multiplier": row['Residential_Multiplier'],
            "commercial_multiplier": row['Commercial_Multiplier'],
            "source": row['Source'],
            "date_acquired": row['Acquisition_Date'],
            "notes": row['Notes']}}
    geojson['features'].append(feature)

with open('mxresult.geojson', 'w') as fp:
    json.dump(geojson, fp)    